#### Prediction challenge - SII course.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



In [2]:
train_data = pd.read_csv('train.csv')
train_data

,Text,Label
0,Quand on évoque aujourd’hui le dérèglement cli...,true
1,Sandra n'est pas une illuminée bien au contrai...,true
2,"Montréal, 10 avril 2008 – Un rapport internati...",true
3,Sur l’échelle de Richter de l’hystérie médiati...,fake
4,Une opinion contre la thèse des réchauffistes....,fake
...,...,...
1943,"La ministre espagnole de l'Environnement, Tere...",true
1944,Par MD On sait que le président de l’imminente...,fake
1945,Voilà plus de 20 ans que Jacques et Albert sou...,true
1946,Climat. La Cop26 de Glasgow sonnera-t-elle la ...,true


In [3]:
test_data = pd.read_csv('test.csv')
test_data

,Text,Label
0,Un texte voté dans une relative unanimité. L'A...,NaN
1,"Si vous voulez agir pour le climat, restez prè...",NaN
2,Nantes-Saint-Nazaire. Transition verte : des i...,NaN
3,« Le bouclier qui protège la Terre du rayonnem...,NaN
4,"Rennes. Ecobio, le labo qui utilise la nature ...",NaN
...,...,...
481,"Aujourd’hui, 75 % des terres agricoles dans le...",NaN
482,"Un pan de berge de la Seine transformé en ""dés...",NaN
483,"EDF, Total et Engie : tous les trois obtiennen...",NaN
484,Il souffle un curieux vent sur la France où in...,NaN


In [4]:
from transformers import AutoTokenizer, AutoModel
import torch

# Încarcă tokenizer-ul și modelul mBERT
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModel.from_pretrained("bert-base-multilingual-cased")

# Testare pe un text multilingv
text = "Salut! Ce mai faci? 你好！你怎么样？"
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
outputs = model(**inputs)

print(outputs.last_hidden_state.shape)


c:\Users\Asus ZenBook\Documents\GitHub\prediction_challenge_SII\.fakeNews\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([1, 18, 768])


In [8]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset

# Încarcă mBERT
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

# Încarcă datele folosind pandas
train_df = pd.read_csv("train.csv")  # Replace with your actual train.csv path
test_df = pd.read_csv("test.csv")    # Replace with your actual test.csv path

# Conversia pandas DataFrame în Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenizatorul
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

# Tokenizare
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Eliminăm coloana originală text, dacă nu mai este necesară
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Text"])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["Text"])

# Specificăm coloanele folosite pentru antrenare
tokenized_train_dataset = tokenized_train_dataset.rename_column("Label", "Labels")
tokenized_test_dataset = tokenized_test_dataset.rename_column("Label", "Labels")

# Configurăm argumentele de antrenare
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
    fp16=True,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
)

# Antrenare
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 486/486 [00:00<00:00, 578.68 examples/s]
c:\Users\Asus ZenBook\Documents\GitHub\prediction_challenge_SII\.fakeNews\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`